# <font color="f0f000">R O B O C L A W</font>
$\rightarrow$ advanced practical use<hr>
Written by: Quentin Demory<br>
August 2020 @ SSRS<br>
MIT license<hr>

In [1]:
from roboclaw_3 import Roboclaw

In [2]:
# Open serial port - Rpi
rc = Roboclaw("/dev/ttyACM0", 115200)
rc.Open()

1

```rc.Open()``` returns 1 if properly set up and the roboclaw is found, otherwise 0

Roboclaw used for this exercise is on address 129 (picture to follow of MC2) and I am only using channel 1/ M1 as the other channel is damaged.

In [3]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 10000, 1)

True

With the last command, we travelled from our minima 0 to our maxima 10000, which for the particular motor that I am using represents 5 revolution. A revolution is 2000 pulses and I found that out by trial and error on the BasicMicro software. below we check that the encoders have come close to that value.

In [4]:
rc.ReadEncM1(129)

(1, 9997, 128)

In [5]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 0, 1)

True

In [6]:
rc.ReadEncM1(129)

(1, 4, 130)

In [11]:
rc.ReadMainBatteryVoltage(129)

(1, 237)

Values returned by the sensors are integers representing a .1 decimal inside a tuple. the first value means that we are properly connected, otherwise it would be 0 and the second is the value we are after.

In [41]:
print("Main battery @ {} V".format(rc.ReadMainBatteryVoltage(129)[1]/10))

Main battery @ 23.7 V


Below is a buffer example. running these two lines will send the motor on an acceleration, speed and decceleration that is arbitrary (I found those values confortable related to the max acceleration of 655359, max decceleration of 655359 [default values suggested by BasicMicro] and a speed that is close to the max QPPS of 59439) to 10000 pulses and when and only when that is done, it will back up to 0 again.

In [13]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 10000, 1)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 0, 0)

True

In [14]:
rc.ReadM1VelocityPID(129)

[1, 0.7190093994140625, 0.041595458984375, 0.0, 59437]

In [15]:
rc.ReadM1PositionPID(129)

[1, 78.0634765625, 0.0, 383.791015625, 0, 4, 0, 10000]

Backing up 5 buffers. **REMEMBER** buffers **ONLY** behave that way on the same channel of the same roboclaw.

In [16]:
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 10000, 1)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 0, 0)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 5000, 0)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 7500, 0)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 10000, 0)
rc.SpeedAccelDeccelPositionM1(129, 400000, 50000, 200000, 0, 0)
rc.ReadBuffers(129)

(1, 5, 128)

A value of 128 in position 1 of the ```rc.ReadBuffers()``` means that the buffers have cleared and were executed in the order they came in. notice position 2 of the same tuple was 128 all the time as it represents the buffers for channel 2 and those were never triggered.

In [17]:
rc.ReadBuffers(129)

(1, 128, 128)

In [36]:
rc.ForwardM1(129, 63) # Forward half speed (PWM min 0, max 127)

True

In [38]:
rc.ForwardM1(129, 0) # Forward no speed -> stop

True

In [39]:
rc.ReadEncM1(129)

(1, 200527, 128)

We see above that we stepped out of the position boundary defined within 0 - 10000 pulses. any new position command will bring back the motor within its position range. If below I had asked the motor to travel from 200527 to 200526, it would have travelled to 10000 as that is its preset position setting that is the closed to the ordered value.<br>
Below is a little trick to wait until buffers have cleared.

In [40]:
import time
start_time = time.time()
rc.SpeedAccelDeccelPositionM1(129, 400000, 10000, 200000, 0, 1)
rc.SpeedAccelDeccelPositionM1(129, 0, 0, 0, 0, 0)
time.sleep(0.2)

while rc.ReadSpeedM1(128)[1] != 0:
        time.sleep(0.02)

print("travel back to 0 from a random position: {} s".format(time.time()-start_time))

travel back to 0 from a random position: 3.230962038040161 s


In [42]:
rc.ReadCurrents(129)

(1, 2, 23)

In [43]:
current_read_channel1 = rc.ReadCurrents(129)[1]/10
current_read_channel2 = rc.ReadCurrents(129)[2]/10

print(current_read_channel1)
print(current_read_channel2)

0.2
1.5


In [45]:
temp_channel1 = rc.ReadTemp(129)[1]/10
temp_channel2 = rc.ReadTemp2(129)[1]/10

print(temp_channel1)
print(temp_channel2)

29.1
29.1


In [46]:
rc.ReadError(129) 
# List of error and their hexadecimal representation can be found on page 74 of the roboclaw manual

(1, 0)

In [47]:
rc.comport

'/dev/ttyACM0'

In [48]:
rc.rate

115200

Following are two commands that behave like ```rc.ForwardM1(address, val)``` but using encoder support rather than only PWM.

In [55]:
rc.SpeedM1(129, 20000)

True

In [56]:
rc.SpeedM1(129, 0)

True

In [57]:
rc.ReadEncM1(129)

(1, 553902, 131)

Now we use the same principle but travel 555000 pulses from 553902 to 1133971 which is a little larger than 555000 due to breaking.

In [58]:
rc.SpeedDistanceM1(129, 50000, 555000, 1)

True

In [60]:
rc.ReadEncM1(129)

(1, 1133971, 128)

In [61]:
rc.SetEncM1(129, 0) # Set encoders to any value. Here I set it back to 0.
rc.ReadEncM1(129)

(1, 0, 128)

Following, I am unsuccessfully trying to break out of the position limits

In [62]:
rc.SpeedAccelDeccelPositionM1(129, 500000, 40000, 250000, 12000, 1)

True

In [63]:
rc.ReadEncM1(129)

(1, 9996, 129)

In [64]:
rc.ReadMainBatteryVoltage(129)[1]/10

23.7

In [65]:
rc.SpeedAccelDeccelPositionM1(129, 500000, 40000, 250000, -10000, 1)

True

In [66]:
rc.ReadEncM1(129)

(1, 4, 130)

Lastly, I am here proceeding backwards.

In [67]:
rc.SpeedAccelDistanceM1(129, 400000, -40000, 10000, 1)

True

In [68]:
rc.ReadEncM1(129)

(1, -30013, 130)